In [1]:
import logging

import nshutils as nu
import rich

logging.basicConfig(level=logging.DEBUG)

nu.pretty()

AttributeError: No nshutils attribute pretty

In [6]:
from __future__ import annotations

import os
from pathlib import Path

import mattertune.configs as MC
from mattertune import MatterTuner

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


def hparams():
    hparams = MC.MatterTunerConfig.draft()

    # Model hparams 
    hparams.model = MC.MACEBackboneConfig.draft()
    hparams.model.pretrained_model = "mace-medium"
    hparams.model.ignore_gpu_batch_transform_error = True
    hparams.model.optimizer = MC.AdamWConfig(lr=1.0e-4)

    hparams.model.properties = []
    energy = MC.EnergyPropertyConfig(loss=MC.MAELossConfig())
    hparams.model.properties.append(energy)
    forces = MC.ForcesPropertyConfig(loss=MC.MAELossConfig(), conservative=True)
    hparams.model.properties.append(forces)
    stresses = MC.StressesPropertyConfig(loss=MC.MAELossConfig(), conservative=True)
    hparams.model.properties.append(stresses)

    # Data hparams
    hparams.data = MC.AutoSplitDataModuleConfig.draft()
    hparams.data.dataset = MC.XYZDatasetConfig.draft()
    hparams.data.dataset.src = Path("./data/Perovskite.xyz")
    hparams.data.train_split = 0.8
    hparams.data.batch_size = 2

    # Trainer hparams
    hparams.trainer.additional_trainer_kwargs = {"fast_dev_run": True}

    hparams = hparams.finalize()
    rich.print(hparams)
    return hparams


hp = hparams()
tune_output = MatterTuner(hp).tune()
model, trainer = tune_output.model, tune_output.trainer

MatterTunerConfig(
    data=AutoSplitDataModuleConfig(
        batch_size=2,
        num_workers='auto',
        pin_memory=True,
        dataset=XYZDatasetConfig(
            type='xyz',
            src=PosixPath('data/Perovskite.xyz'),
            down_sample=None,
            down_sample_refill=False
        ),
        train_split=0.8,
        validation_split='auto',
        shuffle=True,
        shuffle_seed=42
    ),
    model=MACEBackboneConfig(
        reset_backbone=False,
        freeze_backbone=False,
        reset_output_heads=True,
        use_pretrained_normalizers=False,
        properties=[
            EnergyPropertyConfig(
                name='energy',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='energy'
            ),
            ForcesPropertyConfig(
                name='forces',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='forces',
                conservative=True
            ),
            StressesPropertyConfig(
                name='stresses',
                dtype='float',
                loss=MAELossConfig(name='mae', reduction='mean'),
                loss_coefficient=1.0,
                type='stresses',
                conservative=True
            )
        ],
        optimizer=AdamWConfig(
            per_parameter_hparams=None,
            name='AdamW',
            lr=0.0001,
            eps=1e-08,
            betas=(0.9, 0.999),
            weight_decay=0.01,
            amsgrad=False
        ),
        lr_scheduler=None,
        ignore_gpu_batch_transform_error=True,
        normalizers={},
        name='mace',
        pretrained_model='mace-medium'
    ),
    trainer=TrainerConfig(
        accelerator='auto',
        strategy='auto',
        num_nodes=1,
        devices='auto',
        precision='32-true',
        deterministic=None,
        max_epochs=None,
        min_epochs=None,
        max_steps=-1,
        min_steps=None,
        max_time=None,
        val_check_interval=None,
        check_val_every_n_epoch=1,
        log_every_n_steps=None,
        gradient_clip_val=None,
        gradient_clip_algorithm=None,
        checkpoint=None,
        early_stopping=None,
        ema=None,
        loggers='default',
        additional_trainer_kwargs={'fast_dev_run': True}
    ),
    recipes=[]
)

/nethome/lkong88/mace/mace/calculators/mace.py:143: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
INFO:root:CUDA version: 12.6, CUDA device: 0
INFO:mattertune.main:The model requires inference_mode to be disabled. Setting inference_mode=False.
INFO: Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 4 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=4)` but please note that multi-GPU inside interactive / notebook environments is considered experimental a

Using Materials Project MACE for MACECalculator with /nethome/lkong88/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Using head Default out of ['Default']
Default dtype float32 does not match model dtype float64, converting models to float32.
Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 

AttributeError: 'NoneType' object has no attribute 'contiguous'

In [ ]:
property_predictor = model.property_predictor()
print(property_predictor)

calculator = model.ase_calculator()
print(calculator)

In [ ]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

print(property_predictor.predict([atoms], model.hparams.properties))

In [ ]:
import ase

# Create a test periodic system
atoms = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
print(atoms)

# Set the calculator
atoms.calc = calculator

# Calculate the energy
energy = atoms.get_potential_energy()
print(energy)

In [ ]:
property_predictor = model.property_predictor()
atoms_1 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms_2 = ase.Atoms(
    "H2O", positions=[[0, 0, 0], [0, 0, 1], [0, 1, 0]], cell=[10, 10, 10], pbc=True
)
atoms = [atoms_1, atoms_2]
predictions = property_predictor.predict(atoms, ["energy", "forces"])
print("ase.Atoms 1 energy:", predictions[0]["energy"])
print("ase.Atoms 1 forces:", predictions[0]["forces"])
print("ase.Atoms 2 energy:", predictions[1]["energy"])
print("ase.Atoms 2 forces:", predictions[1]["forces"])